In [1]:
import h5py
import tensorflow as tf

In [108]:
class generator:
    def __call__(self):
      with h5py.File('data/electron.hdf5','r') as hf2:
        with h5py.File('data/photon.hdf5', 'r') as hf:
            for i in range(len(hf['X'])): # both hf1 and hf2 have same length
                yield hf['X'][i], hf['y'][i]
                yield hf2['X'][i], hf2['y'][i]                

In [91]:
with h5py.File('data/electron.hdf5', 'r') as hf:
  print(hf['X'].shape)

(249000, 32, 32, 2)


In [95]:
ds = tf.data.Dataset.from_generator(
  generator(),
  output_signature=(
    tf.TensorSpec(shape=(32, 32, 2), dtype=tf.float32),
    tf.TensorSpec(shape=(), dtype=tf.int32)
  )
)

In [109]:
iter_ds=iter(ds)
def input_fn():
    feature, label = next(iter_ds)
    return {"x": feature}, label


In [118]:
batched_ds=ds.batch(32)
iter_batched_ds=iter(batched_ds)

In [120]:
feature, label = next(iter_batched_ds)
label

<tf.Tensor: shape=(32,), dtype=int32, numpy=
array([0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1,
       0, 1, 0, 1, 0, 1, 0, 1, 0, 1])>